## Functions and Tools Creation

Two simple math functions to add and multiply two integers

In [1]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

In [2]:
from langchain.tools import StructuredTool

def multiply_tool():
    tool = StructuredTool.from_function(
        name="Multiply the two given integers",
        func=multiply,
        description="Use this tool if you want to multiply two integers")

    return tool

def add_tool():
    tool = StructuredTool.from_function(
        name="Add the two given integers",
        func=add,
        description="Use this tool if you want to add two integers")

    return tool

### Create instance of LLM

In [31]:
from langchain_ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "greedy",
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 300,
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 50,
    
}
project_id = ""
url="https://us-south.ml.cloud.ibm.com"
apikey=""
model_id="ibm-mistralai/mixtral-8x7b-instruct-v01-q"

llm = WatsonxLLM(model_id=model_id,url=url,apikey=apikey,project_id=project_id, params=parameters)

In [32]:
tools = [multiply_tool(), add_tool()]

In [33]:
template = """You should calculate the answer for the given mathematical equation input completely using the tools.
If the input contains +, then use add_tool and if input contains * then use multiply_tool. 
If the input contains both + and *, then first use multiply_tool and then use add_tool for the final answer.

Begin!

           Text: {input}
            """

In [34]:
from langchain.agents.initialize import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_core.prompts.prompt import PromptTemplate
agent = initialize_agent(llm=llm,
                         tools=tools,
                         agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)
prompt = PromptTemplate(input_variables=['input'], template=template)
agent.run(prompt.format(input="what is (5*3) + (8*2)?"))



> Entering new AgentExecutor chain...
Thought: The human wants me to calculate the value of the given mathematical expression. The expression contains both multiplication and addition operations. According to the instructions, I should first perform the multiplication operations and then the addition operation. I will use the "Multiply the two given integers" tool twice and then use the "Add the two given integers" tool once.

Action:
```
{
  "action": "Multiply the two given integers",
  "action_input": {"a": 5, "b": 3}
}
```

Observation:
Observation: 15
Thought: Now I will multiply 8 and 2 using the same tool.

Action:
```
{
  "action": "Multiply the two given integers",
  "action_input": {"a": 8, "b": 2}
}
```

Observation:
Observation: 16
Thought: Finally, I will add the two results using the "Add the two given integers" tool.

Action:
```
{
  "action": "Add the two given integers",
  "action_input": {"a": 15, "b": 16}
}
```

Observation:
Observation: 31
Thought: I know what to 

'The answer is 31.'